In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
import re

In [5]:
def file_to_sentence_list(file_path):
    with open(file_path, 'r') as file:
        text = file.read()

    # Splitting the text into sentences using
    # delimiters like '.', '?', and '!'
    sentences = [sentence.strip() for sentence in re.split(
        r'(?<=[.!?])\s+', text) if sentence.strip()]

    return sentences

file_path = '../data/data.txt'
text_data = file_to_sentence_list(file_path)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
    input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Convert target data to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [6]:
print(X.shape)
print(y.shape)


(1628, 39)
(1628, 687)


In [7]:
# Define the model
model = Sequential()
model.add(Embedding(total_words, 10,
                    input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',
optimizer='adam', metrics=['accuracy'])


2026-01-29 10:04:42.619497: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2026-01-29 10:04:42.619775: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2026-01-29 10:04:42.619799: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2026-01-29 10:04:42.620211: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-01-29 10:04:42.620612: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
# Train the model
model.fit(X, y, epochs=500, verbose=1)

Epoch 1/500


2026-01-29 10:04:57.678319: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


51/51 [==============================] - 4s 39ms/step - loss: 6.2291 - accuracy: 0.0498
Epoch 2/500
51/51 [==============================] - 1s 26ms/step - loss: 5.7828 - accuracy: 0.0553
Epoch 3/500
51/51 [==============================] - 1s 25ms/step - loss: 5.7243 - accuracy: 0.0485
Epoch 4/500
51/51 [==============================] - 1s 23ms/step - loss: 5.7024 - accuracy: 0.0565
Epoch 5/500
51/51 [==============================] - 1s 24ms/step - loss: 5.6724 - accuracy: 0.0528
Epoch 6/500
51/51 [==============================] - 1s 21ms/step - loss: 5.6266 - accuracy: 0.0565
Epoch 7/500
51/51 [==============================] - 1s 22ms/step - loss: 5.5790 - accuracy: 0.0571
Epoch 8/500
51/51 [==============================] - 1s 22ms/step - loss: 5.5247 - accuracy: 0.0565
Epoch 9/500
51/51 [==============================] - 1s 22ms/step - loss: 5.4733 - accuracy: 0.0651
Epoch 10/500
51/51 [==============================] - 1s 22ms/step - loss: 5.4196 - accuracy: 0.0676
Epoch 11/50

In [15]:
# Generate next word predictions
seed_text = "I dont like pizza "
next_words = 10

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list)
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word

print("Next predicted words:", seed_text)

1/1 [==============================] - 0s 15ms/step
Next predicted words: I dont like pizza  migrated from its mark in the realm of fast food
